# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**

**Fecha**: 21 febrero 2025

**Nombre del Equipo**: ArribaLinux

**Profesor**: Pablo Camarillo Ramirez

152410e2146e148bffc6c65ca433f88635d8a5d959626a04

CONEXIÓN CON EL CLUSTER DE SPARK

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://d9c3cc2bade8:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 23:29:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


IMPORTAR DATOS

In [7]:
import importlib
import ArribaLinux.spark_utils

importlib.reload(ArribaLinux.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = ArribaLinux.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

FILTRAR Y CONTAR CANCIONES POPULARES > 80

In [8]:
from pyspark.sql.functions import col

popular_count = tiktok_df.select("popularity").filter(col("popularity") > 80)
popular_count.count()

1023

CALCULAR PROMEDIO DE DURACIÓN POR GÉNERO

In [9]:
from pyspark.sql.functions import avg

duration_genre_avg = tiktok_df.select("genre", "duration_mins").groupBy("genre").agg(avg("duration_mins"))
duration_genre_avg.show()

+------------------+------------------+
|             genre|avg(duration_mins)|
+------------------+------------------+
|TIKTOK PHILIPPINES|3.2801328435737513|
|      TIKTOK DANCE| 3.015020713916861|
|           _TIKTOK| 3.251196442168827|
|        TIKTOK OPM| 4.257192861885788|
+------------------+------------------+



TOP 5 CANCIONES MÁS ENERGÉTICAS 

In [10]:
from pyspark.sql.functions import desc

top_energetic_songs = tiktok_df.select("track_name", "artist_name", "energy") \
    .orderBy(desc("energy")) \
    .limit(5)
top_energetic_songs.show()


+--------------------+--------------------+------------------+
|          track_name|         artist_name|            energy|
+--------------------+--------------------+------------------+
|       Kiat Jud Dong|         DJ Red Core|0.9990000000000001|
|       Bukan untukku|            DJ Viral|             0.998|
|    Ritmo Envolvente|DJ Menor 7 DJ Men...|             0.995|
|Tante Culik Aku Dong|  Iwansteep Official|             0.995|
|Biarlah Semua Ber...|             DJ Opus|             0.995|
+--------------------+--------------------+------------------+



DURACIÓN TOTAL DE CANCIONES POR PLAYLIST

In [13]:
from pyspark.sql.functions import sum

playlist_total_duration = tiktok_df.select("playlist_name", "duration_mins") \
    .groupBy("playlist_name") \
    .agg(sum("duration_mins").alias("total_duration_mins"))
playlist_total_duration.show()



+--------------------+-------------------+
|       playlist_name|total_duration_mins|
+--------------------+-------------------+
|5IZc3KIVFhjzJ0L2k...|  7.474666666666667|
|08ia51KbTcfs4QVT5...|             4.1485|
|7xVLFuuYdAvcTfcP3...|  9.456433333333333|
|2RBILNmyq8p4fqVWO...|  2.162933333333333|
|6GdDjthxbTGBV9rl2...| 3.3209166666666667|
|7krYEnB1OI1RbnJBa...| 2.0957666666666666|
|1FgPyHX7HruKDL4Tx...|             2.4448|
|62RtxFf9epYNWOUHJ...| 2.6694333333333335|
|5ow0sNF1zSqp71Ix5...| 2.7334833333333335|
|0LlJbV4lyzJYE14YC...| 10.709133333333334|
|6NFKf8vBApSvtzkap...| 3.7074333333333334|
|5P8lyudWE7HQxb4lu...|  4.250666666666667|
|2BgEsaKNfHUdlh97K...|  3.116433333333333|
|7F9vK8hNFMml4GtHs...|  3.173783333333333|
|4vVTI94F9uJ8lHNDW...| 3.3657666666666666|
|2uULRpRtKhCdojXwo...|                2.2|
|1tRlGMHsf21FDo6pj...|            1.79005|
|215fAfwkWtlj30ofd...| 2.3214166666666665|
|3bidbhpOYeV4knp8A...|             4.3057|
|0YFocHKmrMme7Isel...|  4.810383333333333|
+----------

DETENER EL CLUSTER DE SPARK

In [14]:
# Stop the SparkContext
sc.stop()